In [1]:
import pandas as pd
import torch
from torchvision import datasets, transforms
import numpy as np
import sys
import os
from torch.utils.data import Dataset, DataLoader
from torch import from_numpy, tensor


In [2]:
!git clone https://github.com/NicolasMakaroff/PES-metamodeling.git
%cd PES-metamodeling/src/
sys.path.append(os.getcwd())
%cd ../..

Cloning into 'PES-metamodeling'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 3), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.
/Users/nicolasmakaroff/Desktop/S4/PRR/implied-volatility-learning/PES-metamodeling/src
/Users/nicolasmakaroff/Desktop/S4/PRR/implied-volatility-learning


In [3]:
from data_gestion import open_data, norm, create_train_test_set

In [5]:
database = open_data('data/implied-volatility-reduce.csv')   

In [7]:
train_features, train_labels, test_features, test_labels = create_train_test_set(database,0.9,1.,'sigma')  
print(train_features.head())
normed_train_features, normed_test_features = norm(train_features), norm(test_features)
train_dataset = pd.concat([normed_train_features, train_labels], axis=1)
test_dataset = pd.concat([normed_test_features, test_labels], axis=1)
         
train_data = train_dataset.to_numpy(dtype=np.float32) 
test_data = test_dataset.to_numpy(dtype=np.float32)   

              So       tau         r     price
373315  1.075941  0.731160  0.078551  0.145111
459286  1.266984  0.879115  0.020342  0.409147
262398  1.294953  0.710363  0.008480  0.427321
789396  1.067558  0.897104  0.035051  0.317528
383229  0.747197  0.827045  0.034583  0.009318


In [7]:
  """from sklearn.preprocessing import PolynomialFeatures
  from sklearn.metrics import mean_squared_error
  from sklearn import linear_model
  for i in range(7,20):
    polyn = PolynomialFeatures(degree=i)
    x_ = polyn.fit_transform(normed_train_features) #x_ contient les degrés et produits croisés
    #Une fois qu'on a préparé les degrés on peut faire la régression
    clf = linear_model.LinearRegression()
    clf.fit(x_,train_labels)
    y_ = polyn.fit_transform(normed_test_features)
    y1 = clf.predict(y_)
    print('RMSE : {} and degree : {}' .format(mean_squared_error(y1,test_labels),i))"""

"from sklearn.preprocessing import PolynomialFeatures\nfrom sklearn.metrics import mean_squared_error\nfrom sklearn import linear_model\nfor i in range(7,20):\n  polyn = PolynomialFeatures(degree=i)\n  x_ = polyn.fit_transform(normed_train_features) #x_ contient les degrés et produits croisés\n  #Une fois qu'on a préparé les degrés on peut faire la régression\n  clf = linear_model.LinearRegression()\n  clf.fit(x_,train_labels)\n  y_ = polyn.fit_transform(normed_test_features)\n  y1 = clf.predict(y_)\n  print('RMSE : {} and degree : {}' .format(mean_squared_error(y1,test_labels),i))"

In [ ]:
class VolatilityDataset(Dataset):
    """ PES dataset."""

    # Initialize your data, download, etc.
    def __init__(self,data):
        xy = data
        self.len = xy.shape[0]
        self.x_data = from_numpy(xy[:, 0:4])
        self.y_data = from_numpy(xy[:, [-1]])

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


In [ ]:
train_dataset = VolatilityDataset(train_data)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=1024,
                          shuffle=True,
                          num_workers=2)

test_dataset = VolatilityDataset(test_data)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=1024,
                          shuffle=True,
                          num_workers=2)

In [ ]:
from torch import nn, optim
import torch.nn.functional as F

class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 400)
        #self.bn1 = nn.BatchNorm1d(600)
        self.fc2 = nn.Linear(400, 400)
        #self.bn2 = nn.BatchNorm1d(500)
        self.fc3 = nn.Linear(400, 400)
        #self.bn3 = nn.BatchNorm1d(400)
        self.fc4 = nn.Linear(400, 400)
        #self.bn4 = nn.BatchNorm1d(300)
        self.fc5 = nn.Linear(400,400)
        #self.bn5 = nn.BatchNorm1d(200)
        self.fc6 = nn.Linear(400,1)
        
        self.drop = nn.Dropout(p=0.25)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))

        x = self.fc6(x)
        
        return x

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.0)


In [11]:
!git clone https://github.com/awslabs/adatune.git
%cd adatune
!python setup.py install"""

Cloning into 'adatune'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 82 (delta 8), reused 9 (delta 3), pack-reused 60
Unpacking objects: 100% (82/82), done.
/content/adatune
/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [12]:
!pip install git+https://github.com/gbaydin/hypergradient-descent.git

  Cloning https://github.com/gbaydin/hypergradient-descent.git to /tmp/pip-req-build-thkgy4ec
  Running command git clone -q https://github.com/gbaydin/hypergradient-descent.git /tmp/pip-req-build-thkgy4ec
  Created wheel for hypergrad: filename=hypergrad-0.1-cp36-none-any.whl size=8188 sha256=949a8b9451bee3ce4bdd50f388c702ee4d90d1fa034e75d31d549578a6249f4e
  Stored in directory: /tmp/pip-ephem-wheel-cache-1_u8t65n/wheels/fd/a2/2d/63c4e652ba00bf75e1e0e9ce9628cb47c151bd10322a6cf978
Successfully built hypergrad


In [ ]:
#from hypergrad import SGDHD, AdamHD
from adatune.mu_adam import MuAdam
from adatune.network import *
from adatune.utils import *

In [ ]:
model = Regressor()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
#hyper_optim = MuAdam(optimizer, 1e-6, 1000.0, 'store_true', 0.99999, 1e-6, 'cuda')

In [ ]:
epochs = 1500
test_loss_min = np.Inf
steps = 0
model.apply(init_weights)
model.to('cuda')
train_losses, test_losses, learningRate = [], [], []
#vg = ValidationGradient(test_loader, criterion, 'cuda')
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.5)
for e in range(epochs):
    running_loss = 0
    for features, labels in train_loader:
        optimizer.zero_grad() # dé-commenter si on utilise pas Adatune
        features,labels = features.to('cuda'), labels.to('cuda')
        MSE = model(features)
        loss = criterion(MSE, labels)

        # a commenter si on utilise pas adatune
        ###
        """first_grad = ag.grad(loss, model.parameters(), create_graph=True, retain_graph=True)
        hyper_optim.compute_hg(model, first_grad)
        for params, gradients in zip(model.parameters(), first_grad):
            params.grad = gradients
        optimizer.step()
        hyper_optim.hyper_step(vg.val_grad(model))
        clear_grad(model)"""
        ###
        
        # décommenter pour utiliser adamHD
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 2000)
        optimizer.step()
        
        running_loss += loss.item()
        
    else:
        test_loss = 0

        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()
            for features, labels in test_loader:
                features,labels = features.to('cuda'), labels.to('cuda')
                MSE = model(features)
                test_loss += criterion(MSE, labels)

        model.train()        
        train_losses.append(running_loss/len(train_loader))
        test_loss = test_loss/len(test_loader)
        test_losses.append(test_loss/len(test_loader))
        learningRate.append(optimizer.param_groups[0]['lr'])
        #scheduler.step(test_loss)
        scheduler.step()
        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(np.log(running_loss/len(train_loader))),
              "Test Loss: {:.3f}.. ".format(torch.log(test_loss)),
              "Learning rate: {}..".format(optimizer.param_groups[0]['lr']))
        # save model if validation loss has decreased
        if test_loss <= test_loss_min:
            print('Validation loss decreased ({} --> {}).  Saving model ...'.format(test_loss_min,test_loss))
            torch.save(model.state_dict(), 'model5Percent.pt')
            test_loss_min = test_loss

Epoch: 1/1500..  Training Loss: -5.297..  Test Loss: -6.495..  Learning rate: 0.001..
Validation loss decreased (inf --> 0.0015109744854271412).  Saving model ...
Epoch: 2/1500..  Training Loss: -6.444..  Test Loss: -6.629..  Learning rate: 0.001..
Validation loss decreased (0.0015109744854271412 --> 0.001321165356785059).  Saving model ...
Epoch: 3/1500..  Training Loss: -6.536..  Test Loss: -6.565..  Learning rate: 0.001..
Epoch: 4/1500..  Training Loss: -6.581..  Test Loss: -6.821..  Learning rate: 0.001..
Validation loss decreased (0.001321165356785059 --> 0.0010905048111453652).  Saving model ...
Epoch: 5/1500..  Training Loss: -6.628..  Test Loss: -6.569..  Learning rate: 0.001..
Epoch: 6/1500..  Training Loss: -6.643..  Test Loss: -6.877..  Learning rate: 0.001..
Validation loss decreased (0.0010905048111453652 --> 0.0010315345134586096).  Saving model ...
Epoch: 7/1500..  Training Loss: -6.642..  Test Loss: -6.824..  Learning rate: 0.001..
Epoch: 8/1500..  Training Loss: -6.683

The result obtained are a little better than the one reached by the paper only using uniform random sampling. 

![best_result](images/resultat-implied-volatility-BS.png)

The model is save under the name `modelvolatilityBS.pt`.